In [0]:
# Step 1: Load raw data lazily
raw_df = spark.read.option("header", True).csv("dbfs:/mnt/raw/sales.csv")

# Step 2: Transformations (lazy)
filtered_df = raw_df.filter(raw_df["amount"] > 100)
converted_df = filtered_df.withColumn("amount_usd", filtered_df["amount"].cast("double") * 1.1)
summary_df = converted_df.groupBy("region").agg({"amount_usd": "sum"})

# Step 3: Cache intermediate result (optional)
summary_df.cache()

# Step 4: Action - show result
summary_df.show()

# Step 5: Action - save to Delta Lake
summary_df.write.format("delta").mode("overwrite").save("/mnt/curated/sales_summary")
